# Unzip files

In [15]:
import zipfile
zip = zipfile.ZipFile('resampled_data.zip')
zip.extractall()

# Create noisy perturbation

In [5]:
import librosa
import json
from os import listdir
from os.path import isfile, join

import torch
import IPython.display as ipd
import os
import soundfile as sf
# Import the data augmentation component from ASR collection
from nemo.collections.asr.parts import perturb, segment
def load_audio(filepath) -> segment.AudioSegment:
    sample_segment = segment.AudioSegment.from_file(filepath, target_sr=sr)
    return sample_segment

sr = 16000

aug_funcs = []
noise_manifest_path = "noise_files/noise_manifest.jsonl"
   
noise = perturb.NoisePerturbation(manifest_path=noise_manifest_path,
                                          min_snr_db=3, max_snr_db=3,
                                          max_gain_db=200.0)

aug_funcs = [noise, noise, noise, noise, noise, noise, noise, noise, noise, noise, noise, noise]

[NeMo I 2023-01-25 11:42:30 collections:173] Dataset loaded with 12 files totalling 0.22 hours
[NeMo I 2023-01-25 11:42:30 collections:174] 0 files were filtered totalling 0.00 hours


In [13]:
filepath = os.path.join('/resampled_data/test.wav') 
sample_segment = load_audio(filepath)
ipd.Audio(sample_segment.samples, rate=sr)


In [17]:
sample_segment = load_audio(filepath)
noise.perturb(sample_segment)
ipd.Audio(sample_segment.samples, rate=sr)

# Create manifest file

In [34]:
from os import listdir
from os.path import isfile, join
import librosa
import json 
from tqdm import tqdm

resampled_path = "resampled_data/"
onlyfiles = [f for f in listdir(resampled_path) if isfile(join(resampled_path, f))]
training_files =[]

for file in onlyfiles:
    training_files.append(file)
        

manifest_path = 'manifest_file.jsonl'
with open(manifest_path, 'w', encoding="utf-8") as fout:
    for file in tqdm(training_files):
        duration = librosa.core.get_duration(filename=resampled_path+file)
        metadata = {
            "audio_filepath": resampled_path+file,
            "duration": duration,
            "text": (file.split('.wav')[0].strip()).lower()
        }
        json.dump(metadata, fout, ensure_ascii=False)
        fout.write('\n')
        

100%|██████████| 265/265 [00:00<00:00, 2505.34it/s]


# Delete files

In [33]:
import wave, os, glob
path = 'resampled_data'
for filename in glob.glob(os.path.join(path, '*.wav')):
    if "noise" in filename:
        os.remove(filename)

# Create noisy sound samples and new augmented manifest file

In [35]:
manifest_path = 'manifest_file.jsonl'
aug_manifest_path = 'augmented_manifest_file.jsonl'

with open(manifest_path, 'r') as fin:
    with open(aug_manifest_path, 'w', encoding="utf-8") as fout:

        for line in tqdm(fin):
            original_file_json = json.loads(line)
            json.dump(original_file_json, fout, ensure_ascii=False)
            fout.write('\n')

            for i,aug_func in enumerate(aug_funcs):

                sample_segment = load_audio(original_file_json['audio_filepath'])
                aug_func.perturb(sample_segment)

                 # add a new line to the manifest
                new_filepath = f"{original_file_json['audio_filepath'][:-4]}_noise_aug{i}.wav"
                sf.write(new_filepath, sample_segment.samples, sr)

                new_json = original_file_json.copy()
                new_json['audio_filepath'] = new_filepath

                duration = librosa.core.get_duration(filename=new_filepath)
                new_json['duration'] = duration

                json.dump(new_json, fout, ensure_ascii=False)
                fout.write('\n')

265it [00:20, 13.13it/s]
